## Example

In [1]:
import numpy as np
import pandas as pd

from pyrb import EqualRiskContribution, RiskBudgeting

In [ ]:
# get a covariance matrix of an asset universe
covariance_matrix = (
    pd.read_csv("data.csv", sep=";", index_col=0).pct_change().cov() * 260
)

covariance_matrix

,US BONDS 10Y,GERMAN BONDS 10Y,S&P 500,EUROSTOXX 50,NIKKEI,MSCI Emerging,Commodities (CRB),Iboxx HY US,Iboxx HY EUR,Emerging Debt
US BONDS 10Y,0.004116,0.002149,-0.003859,-0.005064,-0.004085,-0.003381,-0.001574,-0.000879,-0.000245,0.000484
GERMAN BONDS 10Y,0.002149,0.003150,-0.002809,-0.005692,-0.003468,-0.003604,-0.001310,-0.000996,-0.000262,0.000280
S&P 500,-0.003859,-0.002809,0.042571,0.030959,0.029899,0.024753,0.008173,0.009952,0.000856,0.004332
EUROSTOXX 50,-0.005064,-0.005692,0.030959,0.064347,0.027036,0.031997,0.011019,0.009402,0.003169,0.006211
NIKKEI,-0.004085,-0.003468,0.029899,0.027036,0.060668,0.031786,0.009928,0.010870,0.001777,0.003836
MSCI Emerging,-0.003381,-0.003604,0.024753,0.031997,0.031786,0.058415,0.014405,0.011145,0.003468,0.007879
Commodities (CRB),-0.001574,-0.001310,0.008173,0.011019,0.009928,0.014405,0.031496,0.005023,0.001489,0.002312
Iboxx HY US,-0.000879,-0.000996,0.009952,0.009402,0.010870,0.011145,0.005023,0.011670,0.001523,0.002549
Iboxx HY EUR,-0.000245,-0.000262,0.000856,0.003169,0.001777,0.003468,0.001489,0.001523,0.004500,0.001282
Emerging Debt,0.000484,0.000280,0.004332,0.006211,0.003836,0.007879,0.002312,0.002549,0.001282,0.008640


#### Solving the ERC problem

In [3]:
ERC = EqualRiskContribution(covariance_matrix)
ERC.solve()

 The optimal solution that gives equal risk contributions is:

In [9]:
optimal_weights = ERC.x
risk_contributions = ERC.get_risk_contributions(scale=False)
risk_contributions_scaled = ERC.get_risk_contributions()

COLUMNS = ["optimal weights", "risk contribution", "risk contribution (scaled)"]

allocation = pd.DataFrame(
    np.concatenate(
        [[optimal_weights, risk_contributions, risk_contributions_scaled]]
    ).T,
    index=covariance_matrix.index,
    columns=COLUMNS,
)

allocation_pct = allocation.copy()
allocation_pct["optimal weights"] = (allocation_pct["optimal weights"] * 100).round(2)
allocation_pct["risk contribution"] = (allocation_pct["risk contribution"] * 100).round(2)
allocation_pct["risk contribution (scaled)"] = (allocation_pct["risk contribution (scaled)"] * 100).round(2)
allocation_pct

,optimal weights,risk contribution,risk contribution (scaled)
US BONDS 10Y,22.45,0.42,10.0
GERMAN BONDS 10Y,27.51,0.42,10.0
S&P 500,3.92,0.42,10.0
EUROSTOXX 50,3.59,0.42,10.0
NIKKEI,3.43,0.42,10.0
MSCI Emerging,2.99,0.42,10.0
Commodities (CRB),5.46,0.42,10.0
Iboxx HY US,7.11,0.42,10.0
Iboxx HY EUR,15.28,0.42,10.0
Emerging Debt,8.25,0.42,10.0


#### Solving the risk budgeting problem


Now we want the risk contributions equal to specific budgets

In [6]:
budgets = [0.1, 0.1, 0.1, 0.2, 0.2, 0.05, 0.05, 0.05, 0.05, 0.1]
RB = RiskBudgeting(covariance_matrix, budgets)
RB.solve()

In [ ]:
optimal_weights = RB.x
risk_contributions = RB.get_risk_contributions(scale=False)
risk_contributions_scaled = RB.get_risk_contributions()

allocation = pd.DataFrame(
    np.concatenate(
        [[optimal_weights, risk_contributions, risk_contributions_scaled]]
    ).T,
    index=covariance_matrix.index,
    columns=COLUMNS,
)

allocation_pct = allocation.copy()
allocation_pct["optimal weights"] = (allocation_pct["optimal weights"] * 100).round(2)
allocation_pct["risk contribution"] = (allocation_pct["risk contribution"] * 100).round(2)
allocation_pct["risk contribution (scaled)"] = (allocation_pct["risk contribution (scaled)"] * 100).round(2)
allocation_pct

,optimal weights,risk contribution,risk contribution (scaled)
US BONDS 10Y,24.50,0.45,10.0
GERMAN BONDS 10Y,30.47,0.45,10.0
S&P 500,3.79,0.45,10.0
EUROSTOXX 50,6.41,0.89,20.0
NIKKEI,6.18,0.89,20.0
MSCI Emerging,1.64,0.22,5.0
Commodities (CRB),3.57,0.22,5.0
Iboxx HY US,4.19,0.22,5.0
Iboxx HY EUR,10.34,0.22,5.0
Emerging Debt,8.91,0.45,10.0
